In [ ]:
# Function (tools) calling

In [1]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

In [2]:
from devtools import debug
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool

from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

In [ ]:
debug("hello")

### Step 1 : Ask LLM to return an instantiated function  

In [4]:
llm = get_llm("mistral_large_edenai")
#llm = get_llm("gpt_4_azure")

messages = []


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    print("'add' tool called")
    return a + b


llm_with_tools = llm.bind_tools([add])
prompt = def_prompt(
    "Use the provided functions  to compute what is 11 + 12; execute the function "
)
chain = prompt | llm_with_tools
ai_msg = chain.invoke({})

debug(ai_msg)

2024-07-18 10:05:30.570 | INFO     | python.ai_core.llm:get_llm:418 - get LLM:'mistral_large_edenai' -configurable: False - streaming: False


/tmp/ipykernel_147259/4210326458.py:26 <module>
    ai_msg: AIMessage(
        content='',
        response_metadata={
            'mistral': {
                'generated_text': '',
                'message': [
                    {
                        'role': 'user',
                        'message': '',
                        'tools': [
                            {
                                'name': 'add',
                                'description': (
                                    'Adds a and b.\n'
                                    '\n'
                                    '    Args:\n'
                                    '        a: first int\n'
                                    '        b: second int'
                                ),
                                'parameters': {
                                    'type': 'object',
                                    'properties': {
                                        'a': {
                         

AIMessage(content='', response_metadata={'mistral': {'generated_text': '', 'message': [{'role': 'user', 'message': '', 'tools': [{'name': 'add', 'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}}], 'tool_calls': None}, {'role': 'assistant', 'message': '', 'tools': None, 'tool_calls': [{'id': '1OU2LnoTl', 'name': 'add', 'arguments': '{"a": 11, "b": 12}'}]}], 'status': 'success', 'cost': 0.003384}}, id='run-9fa05e17-0a92-4e0e-a746-cd20e366bc36-0', tool_calls=[{'name': 'add', 'args': {'a': 11, 'b': 12}, 'id': '1OU2LnoTl', 'type': 'tool_call'}])

In [5]:
debug(llm_with_tools)

/tmp/ipykernel_147259/3318825891.py:1 <module>
    llm_with_tools: RunnableBinding(
        bound=ChatEdenAI(
            provider='mistral',
            model='large-latest',
            max_tokens=2048,
            edenai_api_key=SecretStr('**********'),
        ),
        kwargs={
            'available_tools': [
                {
                    'name': 'add',
                    'description': (
                        'Adds a and b.\n'
                        '\n'
                        '    Args:\n'
                        '        a: first int\n'
                        '        b: second int'
                    ),
                    'parameters': {
                        'type': 'object',
                        'properties': {
                            'a': {
                                'type': 'integer',
                            },
                            'b': {
                                'type': 'integer',
                            },
           

RunnableBinding(bound=ChatEdenAI(provider='mistral', model='large-latest', max_tokens=2048, edenai_api_key=SecretStr('**********')), kwargs={'available_tools': [{'name': 'add', 'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}}], 'tool_choice': None})

In [ ]:
# Step 2 : Execute the function, and send result to LLM

In [6]:
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

'add' tool called


IndexError: list index out of range

###  Other method : Use a tool calling agent

In [7]:
import math

from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def exponentiate(x: float, y: float) -> float:
    """Calculate the power of a number. Return x**y (w to the power of y)"""
    print("exponentiate")
    return math.pow(x, y)


# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Math Assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add, exponentiate]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100^3"})
debug(r)



> Entering new AgentExecutor chain...


/home/tcl/.cache/pypoetry/virtualenvs/genai-blueprint-2X6HL8i2-py3.10/lib/python3.10/site-packages/langchain_community/chat_models/edenai.py:610: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")



Invoking: `exponentiate` with `{'x': 100, 'y': 3}`


exponentiate
1000000.0
Invoking: `add` with `{'a': 12, 'b': 1000000}`


1000012

/home/tcl/.cache/pypoetry/virtualenvs/genai-blueprint-2X6HL8i2-py3.10/lib/python3.10/site-packages/langchain_community/chat_models/edenai.py:610: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")


The result of 100 to the power of 3 is 1000000.

The sum of 12 and 1000000 is 1000012.

> Finished chain.
/tmp/ipykernel_147259/800412656.py:39 <module>
    r: {
        'input': 'what is 12  + 100^3',
        'output': (
            'The result of 100 to the power of 3 is 1000000.\n'
            '\n'
            'The sum of 12 and 1000000 is 1000012.'
        ),
    } (dict) len=2


{'input': 'what is 12  + 100^3',
 'output': 'The result of 100 to the power of 3 is 1000000.\n\nThe sum of 12 and 1000000 is 1000012.'}

In [ ]:
## Call Eden.ai provided tools